# Function Definition

In [48]:
from scipy import optimize
import math
import numpy as np
import pandas as pd
import requests
import io
from six import StringIO 

In [49]:
def F(m, phi_i, d = 0.1*10**(-3), lam = 650*10**(-9)):
    """"
    This function takes as inputs the parameters from our experiment and returns another function, only in terms
    of n, such that it can later be optimized while taking our parameteres as fixed.
    
    Parameters
    -------
    m : the number of fringes counter
    d : thickness of the material plate
    lam : wavelength of the light source
    phi_i : angle through which we rotate the plate, to the normal angle of mirror
    
    Returns
    -------
    funk_to_optimize : a function containing the parameters as fixed, in terms of refractive index 'n'
    
    Notes
    -------
    
    """
    
    def funk_to_optimize(n):
        return m - (2*d)/lam * (n*(1/math.cos(math.asin(math.sin(phi_i)/n)) - 1)
                                + 1 - (math.cos(phi_i - math.asin(math.sin(phi_i)/n))/math.cos(math.asin(math.sin(phi_i)/n))))
    
    return funk_to_optimize

In [50]:
def refrakt(input_data, x0 = 1.4, x1 = 1.5):
    """
    This one takes a list of data and returns the refractive index
    
    Params
    -------
    input_data : experimental data that contains number of fringes 
                 and angle of rotation
    x0 = initial guess for optimization algorithm
    x1 = second guess for optimization algorithm
    
    Returns
    -------
    output_data : list of refractive indices
    
    Example
    -------
    >>>refrakt([[20, 10.4], [40, 14.7]])
    [4.79289677e+34, 4.28057195e+31]
    
    Notes
    -------
    We know that we are measuring glass-like material, so we expect 
    the refractive index to be somewhere between 1.4 and 1.5, hence 
    the default values for the initial two guesses
    
    """
    output_data = []
    
    for arr in input_data:
        
        m = arr[0]
        phi_i = arr[1]
    
        func = F(m, phi_i)
        n = optimize.root_scalar(func, x0 = x0, x1 = x1)
    
        output_data.append(n.root)
        
    return output_data

In [51]:
input_data = [[20, 10.4], [40, 14.7]]
refrakt(input_data)

/Users/ruxi/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/zeros.py:341: RuntimeWarning: Tolerance of 9.58579353658074e+34 reached.
  warnings.warn(msg, RuntimeWarning)
/Users/ruxi/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/zeros.py:341: RuntimeWarning: Tolerance of 8.56114390933573e+31 reached.
  warnings.warn(msg, RuntimeWarning)


[4.792896768290088e+34, 4.280571954662845e+31]

# Data Import and Analysis

In [65]:
url_runB1 = "https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_BR1.csv"
download_runB1 = requests.get(url_runB1).content
runB1 = pd.read_csv(io.StringIO(download_runB1.decode('utf-8')))

url_runB2 = "https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_BR2.csv"
download_runB2 = requests.get(url_runB2).content
runB2 = pd.read_csv(io.StringIO(download_runB2.decode('utf-8')))

url_runB3 = "https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_BR3.csv"
download_runB3 = requests.get(url_runB3).content
runB3 = pd.read_csv(io.StringIO(download_runB3.decode('utf-8')))

url_runW1 = "https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_WR1.csv"
download_runW1 = requests.get(url_runW1).content
runW1 = pd.read_csv(io.StringIO(download_runW1.decode('utf-8')))

url_runW2 = "https://raw.githubusercontent.com/diodeamy/Waves-And-Optics/main/data/data_WR2.csv"
download_runW2 = requests.get(url_runW2).content
runW2 = pd.read_csv(io.StringIO(download_runW2.decode('utf-8')))

In [66]:
data_runs = [[runB1], [runB2], [runB3], [runW1], [runW2]]

In [69]:
runB1

,fringes seen,angle
0,2,1
1,3,2
2,4,3
3,5,4
4,6,5
5,8,6
6,10,7
7,12,8
8,14,9
9,17,10


In [67]:
data_runs

[[    fringes seen  angle
  0              2      1
  1              3      2
  2              4      3
  3              5      4
  4              6      5
  5              8      6
  6             10      7
  7             12      8
  8             14      9
  9             17     10
  10            20     11
  11            23     12
  12            26     13
  13            31     14
  14            36     15
  15            41     16
  16            45     17
  17            50     18
  18            55     19
  19            62     20
  20            69     21
  21            75     22
  22            83     23
  23            93     24
  24           103     25
  25           114     26
  26           126     27
  27           137     28
  28           149     29
  29           162     30],
 [    fringes seen  angle
  0              2      1
  1              3      2
  2              4      3
  3              5      4
  4              6      5
  5              8      6
  6       

In [68]:
refrakt(data_runs)

IndexError: list index out of range